In [ ]:
#                                           # Version Python 3.11.1
import pandas as pd                         # Version 2.0.1
import numpy as np                          # Version 1.24.3
from sqlalchemy import create_engine, text  # Version 2.0.13
from bs4 import BeautifulSoup               # Version 4.12.2
import re
print("fertig")

In [ ]:
## SQL-Server Verbindung erstellen
#Angaben zum SQL-Server
host=""
database=""
user=""
password=""
tabellen_name='jobs.daten_aufgearbeitet' # die aufgearbeitet SQL-Tabelle

# Erstelle eine SQL-Verbindung mit der Datenbank
connection = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}/{database}')

# Daten aus der SQL-Datenbank holen
daten_roh = pd.read_sql_query("SELECT id, seite, seiten_inhalt_html FROM jobs.aktuell", connection)
print("fertig")

In [ ]:
# Suchbegriffe laden
suchbegriffe = pd.read_excel(r"C:\Daten@Server\Python\Block 9.2 - Praxisprojekt\Jobtitel.xlsx",
                            sheet_name='Alle')
print("fertig")

In [ ]:
# die eingelesenen Daten aufteilen nach den Stellenbörsen
daten_stepstone=daten_roh[daten_roh["seite"] == 'stepstone'].reset_index().drop(columns='index')
daten_indeed=daten_roh[daten_roh["seite"] == 'indeed'].reset_index().drop(columns='index')
daten_monster=daten_roh[daten_roh["seite"] == 'monster'].reset_index().drop(columns='index')
daten_linkedin=daten_roh[daten_roh["seite"] == 'linkedin'].reset_index().drop(columns='index')

In [ ]:
## Extrahieren der Daten für Stepstone
# Definition von leeren Listen, in die die extrahierten Daten gespeichert werden sollen
Ort_liste = []                   # für den ort der Stelle
Beruf_liste = []                 # den Jobtitel
Firma_liste = []                 # Die Firma
anstellungsart_liste = []        # für Teilzeit, Vollzeit, etc.
Beschreibungs_liste=[]           # Stellenbeschreibung, Bewerberanforderung, etc.
ID_liste=[]                      # ID aus der Datenbank

print(f'{len(daten_stepstone)} daten_stepstone: {round(len(daten_stepstone)/len(daten_roh)*100,2)} %')

# Iteration über den Inhalt der HTML-Seiten
for i_1, eintrag in enumerate(daten_stepstone.seiten_inhalt_html, start=0):
    
    # Erzeugung eines BeautifulSoup-Objekts aus dem HTML-Eintrag
    soup = BeautifulSoup(eintrag, 'html.parser')
    
    # Herausfiltern der Kopfdaten (Ort, Anstellungsart, etc.)
    kopfdaten=soup.find_all(class_='listing-content-provider-1kd9swq')[0].text.replace('Map marker icon',';;Map marker icon').replace('Briefcase icon',';;Briefcase icon').replace('Clock icon',' ').replace('Calendar icon',';;Calendar icon').replace('Document icon',';;Document icon')\
                                                                                .replace('Alarm clock icon',';;Alarm clock icon').replace('Sei einer der ersten Bewerber',';;Sei einer der ersten Bewerber')\
                                                                                .replace("(m/w/d)", "").replace("(m/w/div.)", "").replace("(m/w/x)", "").replace("(m/f/diverse)", "").replace("(all Genders)", "").replace("(all genders)", "")\
                                                                                .replace("(f/m/x)", "").replace("(w/m/d)", "").replace("(w/m/div.)", "").replace("(mensch)", "").replace("(mwd)", "").replace("(alle Menschen)", "")\
                                                                                .replace("(m/f/d)", "").replace("(M/W/D)", "").replace("(m/f/x)", "").replace("(f/m/d)", "").replace("(w/m/div)", "").replace("(Mensch)", "")\
                                                                                .replace("((m/w/divers))", "").replace("(f/m/div.)", "").replace("(d/m/w)", "").replace("(w/d/m)", "").replace("(w/m/x)", "").replace("(d/f/m)", "")\
                                                                                .replace("(m|w|d)", "").replace("(m/w/div)", "").replace("(w|m|d)", "").replace("(m/w/divers)", "").replace("(m | f | d)", "").replace("(W/M/D)", "")\
                                                                                .replace("m/w/d", "").replace("(gn)", "").replace("m/w/x", "").replace("(d/w/m)", "").replace("(M/F/D)", "").replace("(m,w,d)", "").replace("(w/m/d/x)", "")\
                                                                                .replace("\n", " ")
                                                                                
    # Splitten des Strings
    kopfdaten_liste=kopfdaten.split(";;")
    
    # Entfernen des Teils "Calendar icon" aus der Liste
    for i_2, element in enumerate(kopfdaten_liste):
        if element.startswith("Calendar icon"):
            kopfdaten_liste[i_2] = ""

    # Entfernen leerer Elemente aus der Liste
    if '' in kopfdaten_liste:
        kopfdaten_liste.remove('')
    
    # es werden die gängisten Rechtsformen der Firmen sowie die eindeutige Firmen mit Trennzeichen versehen
    firma_und_beruf=kopfdaten_liste[0].replace("GmbH & Co. KG", "Maker1").replace("AG & Co. KG", "Maker2").replace("(GmbH & Co KG)", "Maker3").replace("Group SE", "Maker4")\
                                      .replace("AG & Co. KG", "Maker2")\
                                      .replace(" SE"," SE;;").replace("AG","AG;;").replace("KG","KG;;").replace("GmbH","GmbH;;").replace("OHG","OHG;;").replace('Personalberatung', 'Personalberatung;;')\
                                      .replace("GbR","GbR;;").replace("Unternehmensgruppe","Unternehmensgruppe;;").replace("Gruppe","Gruppe;;").replace("Group","Group;;").replace('eG','eG;;')\
                                      .replace('Versicherungsgruppe','Versicherungsgruppe;;').replace('gesellschaft mbH','gesellschaft mbH;;')\
                                      .replace('1&1', '1&1;;').replace('BARMER', 'BARMER;;').replace('ZEISS', 'ZEISS;;').replace(' Bundeskriminalamt', ' Bundeskriminalamt;;').replace('TARGOBANK', 'TARGOBANK;;')\
                                      .replace('Vonovia', 'Vonovia;;').replace('Uniper', 'Uniper;;')\
                                      .replace("Maker1","GmbH & Co. KG;;").replace("Maker2","AG & Co. KG;;").replace("Maker3","(GmbH & Co KG);;").replace("Maker4","Group SE;;")\
                                      .split(';;')
    # umwandeln von string in Liste
    angaben=kopfdaten_liste[1:]
    
    ## Listen befüllen
    # extrkation der Firma
    Firma_liste.append(firma_und_beruf[0])
    
    # extraktion des Jobtitels (Beruf)
    try:
        Beruf_liste.append(firma_und_beruf[1])
    except:
        Beruf_liste.append(np.nan)
        
    for angabe in angaben:
        # extraktion der Ortes
        if angabe.startswith('Map marker icon'):
            Ort_liste.append(angabe.replace('Map marker icon', ''))
            
        if angabe.startswith('Briefcase icon'):
            # extraktion des Anstellungsart (Teilzeit, Vollzeit, etc.)
            anstellungsart_liste.append(angabe.replace('Briefcase icon', ''))

    # extraktion der Beschreibung
    try:
        Beschreibungs_liste.append(soup.find_all(class_='listing-content-provider-7mz0xu')[1].text
                                   .split("Gefällt dir der Job?")[0])
    except:
        Beschreibungs_liste.append(np.nan)
    
    ID_liste.append(daten_stepstone.id[i_1])
    
    # auffüllen der Listen mit numpy.nan
    Beruf_liste += [np.nan] * (len(ID_liste) - len(Beruf_liste))
    Firma_liste += [np.nan] * (len(ID_liste) - len(Firma_liste))
    Ort_liste += [np.nan] * (len(ID_liste) - len(Ort_liste))
    anstellungsart_liste += [np.nan] * (len(ID_liste) - len(anstellungsart_liste))
    Beschreibungs_liste += [np.nan] * (len(ID_liste) - len(Beschreibungs_liste))
    
    # DataFrame erstellen
    df_daten_stepstone = pd.DataFrame({"id": ID_liste,
                                       "seite": 'stepstone',
                                       "beruf": Beruf_liste,
                                       "firma": Firma_liste,
                                       "ort": Ort_liste,
                                       "anstellungsart": anstellungsart_liste,
                                       "beschreibung": Beschreibungs_liste})
print("fertig")

In [ ]:
## Extrahieren der Daten für Indeed
# Definition von leeren Listen, in die die extrahierten Daten gespeichert werden sollen
Ort_liste = []                   # für den ort der Stelle
Beruf_liste = []                 # den Jobtitel
Firma_liste = []                 # Die Firma
anstellungsart_liste = []        # für Teilzeit, Vollzeit, etc.
Beschreibungs_liste=[]           # Stellenbeschreibung, Bewerberanforderung, etc.
ID_liste=[]                      # ID aus der Datenbank

print(f'{len(daten_indeed)} daten_indeed: {round(len(daten_indeed)/len(daten_roh)*100,2)} %')

for i_1, eintrag in enumerate(daten_indeed.seiten_inhalt_html):
    soup = BeautifulSoup(eintrag, 'html.parser')

    # extraktion der Firma
    Firma_liste.append(soup.find_all(class_='css-1cjkto6 eu4oa1w0')[0].text.replace("\n","").replace("  ",""))
    
    # extraktion des Ortes
    try:
        Ort_liste.append(soup.find_all(class_='css-6z8o9s eu4oa1w0')[0].text.replace("\n","").replace("  ",""))
    except:
        Ort_liste.append(np.nan)
        
    # extraktion der Jobtitel (Beruf)
    Beruf_liste.append(soup.find_all(class_='icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title')[0].text.replace("\n","").replace("  ","")
                                                                                                                      .replace("(m/w/d)", "").replace("(m/w/div.)", "").replace("(m/w/x)", "").replace("(m/f/diverse)", "").replace("(all Genders)", "").replace("(all genders)", "")\
                                                                                                                      .replace("(f/m/x)", "").replace("(w/m/d)", "").replace("(w/m/div.)", "").replace("(mensch)", "").replace("(mwd)", "").replace("(alle Menschen)", "")\
                                                                                                                      .replace("(m/f/d)", "").replace("(M/W/D)", "").replace("(m/f/x)", "").replace("(f/m/d)", "").replace("(w/m/div)", "").replace("(Mensch)", "")\
                                                                                                                      .replace("(m|w|d)", "").replace("(m/w/div)", "").replace("(w|m|d)", "").replace("(m/w/divers)", "").replace("(m | f | d)", "").replace("(W/M/D)", "")\
                                                                                                                      .replace("m/w/d", "").replace("(gn)", "").replace("m/w/x", "").replace("(d/w/m)", "").replace("(M/F/D)", "").replace("(m,w,d)", "").replace("(w/m/d/x)", "")\
                                                                                                                      .replace("\n", " "))
    
    # extraktion der Beschreibung
    try:
        Beschreibungs_liste.append(soup.find_all(class_='jobsearch-jobDescriptionText jobsearch-JobComponent-description')[0].text.replace("\n","").replace("  ",""))
    except:
        Beschreibungs_liste.append(np.nan)
    
    # extraktion des Beschäftigungsverhlätnises (Vollzeit, Teilzeit, etc.)    
    try:
        anstellung_string=soup.find_all(class_='css-4m8ia3 eu4oa1w0')[0].text.replace("\n","").replace("  ","")
        if anstellung_string == []:
            anstellungsart_liste.append(np.nan)
        elif anstellung_string != []:
            if anstellung_string.startswith('Anstellungsart'):
                anstellungsart_liste.append(anstellung_string.replace('Anstellungsart', '')\
                                                             .replace('VollzeitFestanstellung', 'Maker_1').replace('VollzeitTeilzeitBefristeter Vertrag', 'Maker_2').replace('VollzeitTeilzeit', 'Maker_3')\
                                                             .replace('VollzeitTeilzeitBefristeter VertragTeilzeit möglich', 'Maker_4').replace('VollzeitBefristeter Vertrag', 'Maker_5').replace('VollzeitTeilzeitFestanstellung', 'Maker_6')\
                                                             .replace('VollzeitTeilzeitFestanstellungWerkstudent', 'Maker_7').replace('VollzeitTeilzeitPraktikum', 'Maker_8').replace('FestanstellungArbeitnehmerüberlassung', 'Maker_9')\
                                                             .replace('VollzeitFreie Mitarbeit', 'Maker_10').replace('FestanstellungFreie Mitarbeit', 'Maker_11').replace('VollzeitTeilzeitTeilzeit möglich', 'Maker_12')\
                                                             .replace('VollzeitTeilzeitWerkstudent', 'Maker_13')\
                                                             .replace('Maker_1', 'Vollzeit, Festanstellung').replace('Maker_2', 'Vollzeit, Teilzeit, Befristeter Vertrag').replace('Maker_3', 'Vollzeit Teilzeit')\
                                                             .replace('Maker_4', 'Vollzeit, Teilzeit, Befristeter Vertrag, Teilzeit möglich').replace('Maker_5', 'Vollzeit, Befristeter Vertrag').replace('Maker_6', 'Vollzeit, Teilzeit, Festanstellung')\
                                                             .replace('Maker_7', 'Vollzeit, Teilzeit, Festanstellung, Werkstudent').replace('Maker_8', 'Vollzeit, Teilzeit, Praktikum').replace('Maker_9', 'Festanstellung, Arbeitnehmerüberlassung')\
                                                             .replace('Maker_10', 'Vollzeit, Freie Mitarbeit').replace('Maker_11', 'Festanstellung, Freie Mitarbeit').replace('Maker_12', 'Vollzeit, Teilzeit, Teilzeit möglich')\
                                                             .replace('Maker_13', 'Vollzeit, Teilzeit, Werkstudent'))
            elif anstellung_string.startswith('Gehalt'):
                pass
            elif anstellung_string.startswith('Schichten und Arbeitszeiten'):
                pass
            else:
                anstellungsart_liste.append(np.nan)
    except:
        anstellungsart_liste.append(np.nan)

    ID_liste.append(daten_indeed.id[i_1])
    
    # auffüllen der Listen mit numpy.nan
    Beruf_liste += [np.nan] * (len(ID_liste) - len(Beruf_liste))
    Firma_liste += [np.nan] * (len(ID_liste) - len(Firma_liste))
    Ort_liste += [np.nan] * (len(ID_liste) - len(Ort_liste))
    anstellungsart_liste += [np.nan] * (len(ID_liste) - len(anstellungsart_liste))
    Beschreibungs_liste += [np.nan] * (len(ID_liste) - len(Beschreibungs_liste))
    
    # DataFrame erstellen
    df_daten_indeed = pd.DataFrame({"id": ID_liste,
                                    "seite": 'indeed',
                                    "beruf": Beruf_liste,
                                    "firma": Firma_liste,
                                    "ort": Ort_liste,
                                    "anstellungsart": anstellungsart_liste,
                                    "beschreibung": Beschreibungs_liste})
print("fertig")

In [ ]:
## Extrahieren der Daten für LinkedIn
# Definition von leeren Listen, in die die extrahierten Daten gespeichert werden sollen
Ort_liste = []                   # für den ort der Stelle
Beruf_liste = []                 # den Jobtitel
Firma_liste = []                 # Die Firma
anstellungsart_liste = []        # für Teilzeit, Vollzeit, etc.
Beschreibungs_liste=[]           # Stellenbeschreibung, Bewerberanforderung, etc.
ID_liste=[]                      # ID aus der Datenbank

print(f'{len(daten_linkedin)} daten_linkedin: {round(len(daten_linkedin)/len(daten_roh)*100,2)} %')

# Iteration über den Inhalt der HTML-Seiten
for i_1, eintrag in enumerate(daten_linkedin.seiten_inhalt_html, start=0):
    
    #print(type(eintrag))
    # Erzeugung eines BeautifulSoup-Objekts aus dem HTML-Eintrag
    soup = BeautifulSoup(eintrag, 'html.parser')
    
    # extraktion der Firma
    Firma_liste.append(soup.find_all(class_='topcard__flavor')[0].text.replace("\n","").replace("  ",""))
    
    # extraktion des Ortes
    Ort_liste.append(soup.find_all(class_='topcard__flavor')[1].text.replace("\n","").replace("  ",""))
    
    # extraktion des Jobtitel (Beruf)
    Beruf_liste.append(soup.find_all(class_='top-card-layout__title')[0].text.replace("\n","").replace("  ","")
                                                                             .replace("(m/w/d)", "").replace("(m/w/div.)", "").replace("(m/w/x)", "").replace("(m/f/diverse)", "").replace("(all Genders)", "").replace("(all genders)", "")\
                                                                             .replace("(f/m/x)", "").replace("(w/m/d)", "").replace("(w/m/div.)", "").replace("(mensch)", "").replace("(mwd)", "").replace("(alle Menschen)", "")\
                                                                             .replace("(m/f/d)", "").replace("(M/W/D)", "").replace("(m/f/x)", "").replace("(f/m/d)", "").replace("(w/m/div)", "").replace("(Mensch)", "")\
                                                                             .replace("((m/w/divers))", "").replace("(f/m/div.)", "").replace("(d/m/w)", "").replace("(w/d/m)", "").replace("(w/m/x)", "").replace("(d/f/m)", "")\
                                                                             .replace("(m|w|d)", "").replace("(m/w/div)", "").replace("(w|m|d)", "").replace("(m/w/divers)", "").replace("(m | f | d)", "").replace("(W/M/D)", "")\
                                                                             .replace("m/w/d", "").replace("(gn)", "").replace("m/w/x", "").replace("(d/w/m)", "").replace("(M/F/D)", "").replace("(m,w,d)", "").replace("(w/m/d/x)", "")\
                                                                             .replace("\n", " "))
    
    # extraktion der Beschreibung
    try:
        Beschreibungs_liste.append(soup.find_all(class_='show-more-less-html__markup relative overflow-hidden')[0].text.replace("\n","").replace("  ",""))
    except:
        Beschreibungs_liste.append(np.nan)

    # extraktion des Beschäftigungsverhlätnises (Vollzeit, Teilzeit, etc.)
    try:
        for eintraege in soup.find_all(class_='description__job-criteria-item'):
            eintrag=eintraege.text.replace('\n','').replace(' ','')
            
            if eintrag.startswith('Beschäftigungsverhältnis'):
                anstellungsart_liste.append(eintrag.replace('Beschäftigungsverhältnis', ''))
            elif eintrag.startswith('Branchen'):
                pass
            elif eintrag.startswith('Tätigkeitsbereich'):
                pass
            elif eintrag.startswith('Karrierestufe'):
                pass
    except:
        anstellungsart_liste.append(np.nan)
            
        
    
    ID_liste.append(daten_linkedin.id[i_1])
    
    # auffüllen der Listen mit numpy.nan
    Beruf_liste += [np.nan] * (len(ID_liste) - len(Beruf_liste))
    Firma_liste += [np.nan] * (len(ID_liste) - len(Firma_liste))
    Ort_liste += [np.nan] * (len(ID_liste) - len(Ort_liste))
    anstellungsart_liste += [np.nan] * (len(ID_liste) - len(anstellungsart_liste))
    Beschreibungs_liste += [np.nan] * (len(ID_liste) - len(Beschreibungs_liste))
    
    # DataFrame erstellen
    df_daten_linkedin = pd.DataFrame({"id": ID_liste,
                                      "seite": 'linkedin',
                                      "beruf": Beruf_liste,
                                      "firma": Firma_liste,
                                      "ort": Ort_liste,
                                      "anstellungsart": anstellungsart_liste,
                                      "beschreibung": Beschreibungs_liste})
print("fertig")

In [ ]:
## Extrahieren der Daten für Monster
# Definition von leeren Listen, in die die extrahierten Daten gespeichert werden sollen
Ort_liste = []                   # für den ort der Stelle
Beruf_liste = []                 # den Jobtitel
Firma_liste = []                 # Die Firma
anstellungsart_liste = []        # für Teilzeit, Vollzeit, etc.
Beschreibungs_liste=[]           # Stellenbeschreibung, Bewerberanforderung, etc.
ID_liste=[]                      # ID aus der Datenbank

print(f'{len(daten_monster)} daten_monster: {round(len(daten_monster)/len(daten_roh)*100,2)} %')

# Iteration über den Inhalt der HTML-Seiten
for i_1, eintrag in enumerate(daten_monster.seiten_inhalt_html, start=0):
    
    # Erzeugung eines BeautifulSoup-Objekts aus dem HTML-Eintrag
    soup = BeautifulSoup(eintrag, 'html.parser')
    
    # extraktion vom Jobtitel (Beruf)
    for class_name_1 in ['cached-bot-header-skeletonstyle__CachedBotJobViewHeaderCompany-sc-rp06gd-5','cached-bot-header-skeletonstyle__CachedBotJobViewHeaderTitle-sc-rp06gd-4','headerstyle__JobViewHeaderTitle-sc-1ijq9nh-5', 'cached-bot-header-skeletonstyle__CachedBotJobViewHeaderTitle-sc-rp06gd-4']:
        if soup.findAll(class_=class_name_1) != []:
            beruf_class_name=class_name_1
            break
            
    beruf=soup.findAll(class_=beruf_class_name)

    try:
        Beruf_liste.append(beruf[0].text.replace("\n","").replace("  ","")\
                                        .replace("(m/w/d)", "").replace("(m/w/div.)", "").replace("(m/w/x)", "").replace("(m/f/diverse)", "").replace("(all Genders)", "").replace("(all genders)", "")\
                                        .replace("(f/m/x)", "").replace("(w/m/d)", "").replace("(w/m/div.)", "").replace("(mensch)", "").replace("(mwd)", "").replace("(alle Menschen)", "")\
                                        .replace("(m/f/d)", "").replace("(M/W/D)", "").replace("(m/f/x)", "").replace("(f/m/d)", "").replace("(w/m/div)", "").replace("(Mensch)", "")\
                                        .replace("((m/w/divers))", "").replace("(f/m/div.)", "").replace("(d/m/w)", "").replace("(w/d/m)", "").replace("(w/m/x)", "").replace("(d/f/m)", "")\
                                        .replace("(m|w|d)", "").replace("(m/w/div)", "").replace("(w|m|d)", "").replace("(m/w/divers)", "").replace("(m | f | d)", "").replace("(W/M/D)", "")\
                                        .replace("m/w/d", "").replace("(gn)", "").replace("m/w/x", "").replace("(d/w/m)", "").replace("(M/F/D)", "").replace("(m,w,d)", "").replace("(w/m/d/x)", "")\
                                        .replace('m / f / d', '').replace('M/W/D', '').replace("\n", " "))
    except:
        Beruf_liste.append(np.nan)

    for class_name_2 in  ['company-name-linkstyles__CompanyLink-sc-1f27k6n-0 dCguCv','company-name-linkstyles__CompanyLink-sc-1f27k6n-0','cached-bot-header-skeletonstyle__CachedBotJobViewHeaderCompany-sc-rp06gd-5','headerstyle__JobViewHeaderCompany-sc-1ijq9nh-6','cached-bot-header-skeletonstyle__CachedBotJobViewHeaderCompany-sc-rp06gd-5']:
        if soup.findAll(class_=class_name_2) != []:
            firma_class_name=class_name_2
            break
        
    # extraktion der Firma (zukünftigen Arbeitgebers)
    firma=soup.findAll(class_=firma_class_name)
    
    try:
        Firma_liste.append(firma[0].text)
    except:
        Firma_liste.append(np.nan)
        
    ## extraktion von Ort, Teilezit, Vollzeit, etc.
    # Extraktion der Stellenbeschreibung aus verschiedenen Quellen
    stellenbeschreibung1=soup.find_all(id='details-table')
    stellenbeschreibung2=soup.find_all(class_='detailsstyles__DetailsTable-sc-1deoovj-0')
    stellenbeschreibung3=soup.find_all(class_='cached-bot-shared-skeletonstyle__CachedSkeletonLink-sc-15ar3p6-8')
    stellenbeschreibung4=soup.find_all(class_='cached-bot-fjv-skeletonstyle__CachedSkeletonDescription-sc-gqtazs-0')
    stellenbeschreibung5=soup.find_all(class_='detailsstyles__DetailsTableHeader-sc-1deoovj-1')
    
    # Überprüfe, ob eine Stellenbeschreibung gefunden wurde und extrahiere relevante Informationen
    if stellenbeschreibung1 != []:
        hat_funktioniert=True
        stellenbeschreibung=stellenbeschreibung1[0].text.replace('STELLENBESCHREIBUNG','')\
                                                        .replace('ANSTELLUNGSART',';;ANSTELLUNGSART ').replace('ORT',';;ORT ')\
                                                        .replace('VERÖFFENTLICHT',';;VERÖFFENTLICHT ').replace('GEHALT',';;GEHALT ')
    elif stellenbeschreibung2 != []:
        hat_funktioniert=True
        stellenbeschreibung=stellenbeschreibung2[0].text.replace('STELLENBESCHREIBUNG','')\
                                                        .replace('ANSTELLUNGSART',';;ANSTELLUNGSART ').replace('ORT',';;ORT ')\
                                                        .replace('VERÖFFENTLICHT',';;VERÖFFENTLICHT ').replace('GEHALT',';;GEHALT ')
    elif stellenbeschreibung3 != []:
        hat_funktioniert=True
        stellenbeschreibung=stellenbeschreibung3[0].text.replace('STELLENBESCHREIBUNG','')\
                                                        .replace('ANSTELLUNGSART',';;ANSTELLUNGSART ').replace('ORT',';;ORT ')\
                                                        .replace('VERÖFFENTLICHT',';;VERÖFFENTLICHT ').replace('GEHALT',';;GEHALT ')
    elif stellenbeschreibung4 != []:
        hat_funktioniert=True
        stellenbeschreibung=stellenbeschreibung4[0].text.replace('STELLENBESCHREIBUNG','')\
                                                        .replace('ANSTELLUNGSART',';;ANSTELLUNGSART ').replace('ORT',';;ORT ')\
                                                        .replace('VERÖFFENTLICHT',';;VERÖFFENTLICHT ').replace('GEHALT',';;GEHALT ')
    elif stellenbeschreibung5 != []:
        hat_funktioniert=True
        stellenbeschreibung=stellenbeschreibung5[0].text.replace('STELLENBESCHREIBUNG','')\
                                                        .replace('ANSTELLUNGSART',';;ANSTELLUNGSART ').replace('ORT',';;ORT ')\
                                                        .replace('VERÖFFENTLICHT',';;VERÖFFENTLICHT ').replace('GEHALT',';;GEHALT ')
    else:
        hat_funktioniert=False
        anstellungsart_liste.append(np.nan)
        Ort_liste.append(np.nan)
    
    # Teile die Stellenbeschreibung anhand des Trennzeichens ";;"
    parts = stellenbeschreibung.split(";;")

    # Entferne führende und abschließende Leerzeichen von jedem Teil und entferne leere Teile
    parts = [part.strip() for part in parts if part.strip() != ""]

    # Initialisiere Variablen für den Jobtyp und den Ort
    job_type = ""
    location = ""

    # Überprüfe, ob das Aufteilen der Stellenbeschreibung erfolgreich war
    if hat_funktioniert == True:
        # Teile die Stellenbeschreibung erneut anhand des Trennzeichens ";;"
        parts = stellenbeschreibung.split(";;")

        # Entferne führende und abschließende Leerzeichen von jedem Teil und entferne leere Teile
        parts = [part.strip() for part in parts if part.strip() != ""]

        # Iteriere über die Teile der Stellenbeschreibung
        for part in parts:
            # Überprüfe, ob der Teil mit "ANSTELLUNGSART" beginnt
            if part.startswith("ANSTELLUNGSART"):
                # Extrahiere den Jobtyp, indem du "ANSTELLUNGSART " entfernst
                job_type = part.replace("ANSTELLUNGSART ", "")
            # Überprüfe, ob der Teil mit "ORT" beginnt
            elif part.startswith("ORT"):
                # Extrahiere den Ort, indem du "ORT " entfernst
                location = part.replace("ORT ", "")

        # Füge den extrahierten Jobtyp zur Liste anstellungsart_liste hinzu
        if job_type != "":
            anstellungsart_liste.append(job_type)
        else:
            anstellungsart_liste.append(np.nan)

        # Füge den extrahierten Ort zur Liste Ort_liste hinzu
        if location != "":
            Ort_liste.append(location)
        else:
            Ort_liste.append(np.nan)

    ID_liste.append(daten_monster.id[i_1])
    
    # auffüllen der Listen mit numpy.nan
    Beruf_liste += [np.nan] * (len(ID_liste) - len(Beruf_liste))
    Firma_liste += [np.nan] * (len(ID_liste) - len(Firma_liste))
    Ort_liste += [np.nan] * (len(ID_liste) - len(Ort_liste))
    anstellungsart_liste += [np.nan] * (len(ID_liste) - len(anstellungsart_liste))
    Beschreibungs_liste += [np.nan] * (len(ID_liste) - len(Beschreibungs_liste))
    
# DataFrame erstellen
df_daten_monster = pd.DataFrame({"id": ID_liste,
                                 "seite": 'monster',
                                 "beruf": Beruf_liste,
                                 "firma": Firma_liste,
                                 "ort": Ort_liste,
                                 "anstellungsart": anstellungsart_liste,
                                 "beschreibung": Beschreibungs_liste})

#entfernen der Zeilen die in den Spalten 'beruf','firma','ort' NaN-Werte enthalten
print(f'Beruf: {df_daten_monster.beruf.isna().sum()} NaN-Werte')
print(f'Firma: {df_daten_monster.firma.isna().sum()} NaN-Werte')
print(f'Ort: {df_daten_monster.ort.isna().sum()} NaN-Werte')
df_daten_monster.dropna(subset=['beruf','firma','ort'], how="all", inplace=True)
print("fertig")

In [ ]:
## zusammenführen der einzelnen Dataframes
# Verbinde die DataFrames vertikal
df_alle = pd.concat([df_daten_stepstone, df_daten_linkedin, df_daten_monster, df_daten_indeed],
                    axis=0).reset_index().drop(columns='index')
print("fertig")

In [ ]:
print(f'Stepstone:\nLänge: {len(df_daten_stepstone)}\n{df_daten_stepstone.isna().sum()}\n')
print(f'linkedin:\nLänge: {len(df_daten_linkedin)}\n{df_daten_linkedin.isna().sum()}\n')
print(f'monster:\nLänge: {len(df_daten_monster)}\n{df_daten_monster.isna().sum()}\n')
print(f'indeed:\nLänge: {len(df_daten_indeed)}\n{df_daten_indeed.isna().sum()}\n')
print(f'Alle:\nLänge: {len(df_alle)}\n{df_alle.isna().sum()}\n')

In [ ]:
# nach NaN_Werte in Spalte Beruf filtern
df_mit_nan=df_alle[df_alle.beruf.isna()].reset_index().drop(columns='index')
print("fertig")

In [ ]:
# Filterung vom DataFrame mit NaN
# nach den Jobtiteln in der Spalte Firma, da bei stepstone nicht alle Firmen- und Jobtitelbezeichnungn getrennt werden können
id_liste=[]
for i1, eintrag in enumerate(df_mit_nan.firma):
    for i2 in suchbegriffe.Jobtitel:
        pattern=f'.*{i2}.*'
        x=re.findall(pattern, eintrag)
        if x != []:
            id_liste.append(df_mit_nan.id[i1])
print("fertig")

In [ ]:
# DataFrame aus der Liste ID erstellen
df_id=pd.DataFrame({"id":id_liste})

# Die DataFrames "df_id" und "df_mit_nan" zusammenbringen
gefiltert_df_mit_nan=df_id.merge(df_mit_nan, left_on='id', right_on='id').drop_duplicates(subset="beruf")
print("fertig")

In [ ]:
# DataFrame ohne NaN-Werte in der Spalte Beruf
df_ohne_nan=df_alle.dropna(subset='beruf').reset_index().drop(columns='index')
print("fertig")

In [ ]:
# Filterung vom DataFrame ohne NaN
# nach den Jobtiteln in der Spalte Beruf
id_liste=[]
for i1, eintrag in enumerate(df_ohne_nan.beruf):
    for i2 in suchbegriffe.Jobtitel:
        pattern=f'.*{i2}.*'
        x=re.findall(pattern, eintrag)
        if x != []:
            id_liste.append(df_ohne_nan.id[i1])
print("fertig")

In [ ]:
# DataFrame aus der Liste ID erstellen
df_id=pd.DataFrame({"id":id_liste})

# Die DataFrames "df_id" und "df_ohne_nan" zusammenbringen
gefiltert_df_ohne_nan=df_id.merge(df_ohne_nan, left_on='id', right_on='id').drop_duplicates(subset="beruf")
print("fertig")

In [ ]:
# Die DataFrames "df_mit_nan" und "gefiltert_df_ohne_nan" zusammenbringen (vertikal)
# sowie die Spalte seite löschen, da diese nur für die Aufsplittung in die DataFrames mit und ohne NaN benötigt wurde
gefiltert_df_alle=pd.concat([gefiltert_df_mit_nan, gefiltert_df_ohne_nan],
                            axis=0).drop(columns='seite').reset_index().drop(columns='index')
print("fertig")

In [ ]:
# inhalt der SQL-Tabelle löschen
try:
    with connection.connect() as SQL_verbindung:
        sql_befehl=text(f'DELETE FROM {tabellen_name}')
        SQL_verbindung.execute(sql_befehl)
        SQL_verbindung.commit()
    print(f'Die Datenbank "{tabellen_name}" wurde erfolgreich geleert')
except:
    print(f'Konnte die Datenbank "{tabellen_name}" nicht leeren')
    
# DataFrame in SQL-Speichern
try:
    gefiltert_df_alle.to_sql(name=tabellen_name.split('.')[1],
                            schema=tabellen_name.split('.')[0],
                            con=connection, if_exists='append', index=False)
    print(f'{len(gefiltert_df_alle)} in die Datenbank "{tabellen_name}" geschrieben')
except:
    print(f'Die Datenbank "{tabellen_name}" konnte nicht geschrieben werden')

In [ ]:
print(f'        Rohdaten: {len(daten_roh)}\nGefilterte Daten: {len(gefiltert_df_alle)}\n')
print(f'{round(len(gefiltert_df_alle)/len(daten_roh)*100,2)} % verbleiben\n')
display(gefiltert_df_alle)